# BESTLIB - Demo de Gráficos Modulares

Este notebook demuestra el uso de la nueva API modular de BESTLIB con diferentes tipos de gráficos.


## 1. Configuración del Entorno

### Para Google Colab:
1. Sube la carpeta `BESTLIB` completa al directorio `/content/` usando el panel de archivos
2. O clona el repositorio: `!git clone <repo_url>`
3. O especifica la ruta manualmente en la celda siguiente

### Para Jupyter Local:
El código detectará automáticamente la ruta del proyecto.


In [3]:
# Configuración del entorno y detección de Colab
import sys
import os
from pathlib import Path

# Detectar si estamos en Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# CRÍTICO: Remover versiones instaladas de BESTLIB del sys.path
# Esto evita conflictos con versiones antiguas sin estructura modular
sys.path = [p for p in sys.path if 'dist-packages/BESTLIB' not in p and 'site-packages/BESTLIB' not in p]

# OPCIÓN: Especificar ruta manualmente si es necesario
# Descomenta y ajusta la siguiente línea si la detección automática falla:
# MANUAL_PATH = "/content/bestlib"  # Para Colab
# MANUAL_PATH = "/Users/nahiaescalante/Documents/2025/Visualizacion/bestlib"  # Para local

if IN_COLAB:
    print("🔵 Detectado: Google Colab")
    
    # Verificar si hay ruta manual
    if 'MANUAL_PATH' in globals() and MANUAL_PATH:
        root = Path(MANUAL_PATH)
        print(f"📁 Usando ruta manual: {root}")
    else:
        # Buscar BESTLIB en ubicaciones comunes de Colab
        search_paths = [
            Path('/content') / 'bestlib',
            Path('/content') / 'BESTLIB',
            Path.cwd() / 'bestlib',
            Path.cwd() / 'BESTLIB',
            Path('/content') / 'bestlib' / 'BESTLIB',
        ]
        
        root = None
        for search_path in search_paths:
            if (search_path / 'BESTLIB').exists() or (search_path.name == 'BESTLIB' and search_path.exists()):
                if search_path.name == 'BESTLIB':
                    root = search_path.parent
                else:
                    root = search_path
                break
        
        if root is None:
            print("❌ BESTLIB no encontrado automáticamente")
            print("\nPor favor:")
            print("1. Sube la carpeta BESTLIB completa a /content/")
            print("   (debe contener: core/, charts/, data/, layouts/, etc.)")
            print("2. O descomenta y ajusta MANUAL_PATH arriba")
            print(f"3. Directorio actual: {Path.cwd()}")
            print("\nEstructura esperada:")
            print("  /content/bestlib/BESTLIB/")
            print("    core/")
            print("    charts/")
            print("    data/")
            print("    layouts/")
            raise FileNotFoundError("BESTLIB no encontrado. Por favor, sube la carpeta BESTLIB completa.")
        else:
            print(f"✅ Encontrado BESTLIB en: {root}")
    
    # Agregar al INICIO del path para máxima prioridad
    root_str = str(root)
    if root_str in sys.path:
        sys.path.remove(root_str)  # Remover si ya está
    sys.path.insert(0, root_str)  # Insertar al principio
    print(f"📁 Path configurado: {root_str} (prioridad máxima)")
else:
    print("💻 Entorno local detectado")
    
    # Verificar si hay ruta manual
    if 'MANUAL_PATH' in globals() and MANUAL_PATH:
        root = Path(MANUAL_PATH)
        print(f"📁 Usando ruta manual: {root}")
    else:
        # Detectar ruta del proyecto automáticamente
        current = Path.cwd()
        
        # Si estamos en examples/, subir un nivel
        if current.name == 'examples':
            root = current.parent
        else:
            # Buscar el directorio que contiene BESTLIB
            root = current
            while root != root.parent:
                if (root / 'BESTLIB').exists():
                    break
                root = root.parent
            else:
                # Si no se encuentra, usar el directorio actual
                root = current
        
        print(f"📁 Ruta del proyecto detectada: {root}")
    
    root_str = str(root)
    # Remover versiones instaladas del path
    sys.path = [p for p in sys.path if 'dist-packages/BESTLIB' not in p and 'site-packages/BESTLIB' not in p]
    if root_str not in sys.path:
        sys.path.insert(0, root_str)
    
    # Verificar que BESTLIB existe y tiene estructura modular
    bestlib_path = root / 'BESTLIB'
    bestlib_exists = bestlib_path.exists()
    print(f"📁 BESTLIB existe: {bestlib_exists}")
    
    if bestlib_exists:
        # Verificar estructura modular
        required_dirs = ['core', 'charts', 'data']
        missing_dirs = [d for d in required_dirs if not (bestlib_path / d).exists()]
        if missing_dirs:
            print(f"⚠️ Estructura modular incompleta. Faltan: {missing_dirs}")
            print("   Asegúrate de subir la versión modular completa de BESTLIB")
        else:
            print(f"✅ Estructura modular verificada: {required_dirs}")
    
    if not bestlib_exists:
        print(f"❌ BESTLIB no encontrado en: {bestlib_path}")
        print("Por favor, verifica la ruta o especifica MANUAL_PATH")
        raise FileNotFoundError(f"BESTLIB no encontrado en {root}")

# Verificar que podemos importar antes de continuar
print("\n🔍 Verificando imports...")

# Intentar importar
try:
    # Primero verificar que el módulo core existe
    import importlib.util
    core_path = Path(root) / 'BESTLIB' / 'core' / '__init__.py'
    if not core_path.exists():
        raise ImportError(f"Módulo core no encontrado en {core_path}")
    
    # Intentar imports directos desde módulos
    from BESTLIB.charts.registry import ChartRegistry
    from BESTLIB.charts.scatter import ScatterChart
    from BESTLIB.charts.bar import BarChart
    from BESTLIB.charts.histogram import HistogramChart
    from BESTLIB.charts.boxplot import BoxplotChart
    from BESTLIB.charts.heatmap import HeatmapChart
    from BESTLIB.charts.line import LineChart
    from BESTLIB.charts.pie import PieChart
    
    # Intentar importar MatrixLayout desde layouts o matrix
    try:
        from BESTLIB.layouts.matrix import MatrixLayout
    except ImportError:
        from BESTLIB.matrix import MatrixLayout
    
    from BESTLIB.data.preparators import prepare_scatter_data
    from BESTLIB.data.validators import validate_scatter_data
    
    print("✅ Todos los módulos importados correctamente")
    
except ImportError as e:
    print(f"❌ Error al importar: {e}")
    print(f"\n📁 Path actual: {sys.path[:3]}...")  # Mostrar primeros 3 paths
    print(f"📁 Root configurado: {root}")
    print(f"📁 BESTLIB path: {root / 'BESTLIB'}")
    
    # Verificar estructura
    if (root / 'BESTLIB').exists():
        print("\n📂 Estructura encontrada:")
        for item in (root / 'BESTLIB').iterdir():
            if item.is_dir():
                print(f"   ✓ {item.name}/")
            else:
                print(f"   - {item.name}")
    
    print("\n💡 Solución:")
    print("1. En Colab, sube la carpeta BESTLIB COMPLETA (con todas las subcarpetas)")
    print("2. Verifica que la estructura sea:")
    print("   /content/bestlib/BESTLIB/")
    print("     core/")
    print("     charts/")
    print("     data/")
    print("     layouts/")
    print("     ...")
    print("3. O descomenta MANUAL_PATH y especifica la ruta exacta")
    print("4. Reinicia el runtime después de subir los archivos")
    raise

import pandas as pd
import numpy as np

print(f"\n📊 Verificación:")
print(f"  - ChartRegistry: {ChartRegistry is not None}")
print(f"  - MatrixLayout: {MatrixLayout is not None}")
print(f"  - pandas: {pd.__version__}")
print(f"  - numpy: {np.__version__}")


💻 Entorno local detectado
📁 Ruta del proyecto detectada: /Users/nahiaescalante/Documents/2025/Visualizacion/bestlib
📁 BESTLIB existe: True
✅ Estructura modular verificada: ['core', 'charts', 'data']

🔍 Verificando imports...
✅ Todos los módulos importados correctamente


ModuleNotFoundError: No module named 'pandas'

## 2. Generar Datos de Ejemplo


In [ ]:
# Generar datos sintéticos
np.random.seed(42)
n = 200

df = pd.DataFrame({
    'edad': np.random.randint(20, 70, n),
    'salario': np.random.normal(50000, 15000, n),
    'departamento': np.random.choice(['IT', 'Ventas', 'Marketing', 'HR'], n),
    'experiencia': np.random.randint(0, 20, n),
    'productividad': np.random.normal(75, 15, n),
    'ventas': np.random.randint(1000, 10000, n)
})

# Asegurar valores positivos
df['salario'] = df['salario'].clip(lower=20000)
df['productividad'] = df['productividad'].clip(lower=0, upper=100)

print(f"Dataset generado: {len(df)} filas")
print(df.head())


## 3. Scatter Plot - Usando Chart Registry


In [ ]:
# Obtener scatter chart desde registry
scatter = ChartRegistry.get('scatter')

# Generar spec
spec = scatter.get_spec(
    data=df,
    x_col='edad',
    y_col='salario',
    category_col='departamento',
    interactive=True,
    pointRadius=4,
    axes=True
)

# Crear layout y mostrar
layout = MatrixLayout("S")
layout.map("S", spec)
layout.display()


## 4. Bar Chart - Ventas por Departamento


In [ ]:
# Usar BarChart directamente
bar = BarChart()
spec = bar.get_spec(
    data=df,
    category_col='departamento',
    value_col='ventas',
    interactive=True,
    color='steelblue'
)

layout = MatrixLayout("B")
layout.map("B", spec)
layout.display()


## 5. Histograma - Distribución de Salarios


In [ ]:
# Histograma de salarios
hist = HistogramChart()
spec = hist.get_spec(
    data=df,
    value_col='salario',
    bins=20,
    interactive=True
)

layout = MatrixLayout("H")
layout.map("H", spec)
layout.display()


## 6. Boxplot - Salarios por Departamento


In [ ]:
# Boxplot agrupado por departamento
boxplot = BoxplotChart()
spec = boxplot.get_spec(
    data=df,
    category_col='departamento',
    value_col='salario',
    interactive=True
)

layout = MatrixLayout("B")
layout.map("B", spec)
layout.display()


## 7. Pie Chart - Distribución de Departamentos


In [ ]:
# Pie chart de departamentos
pie = PieChart()
spec = pie.get_spec(
    data=df,
    category_col='departamento',
    interactive=True
)

layout = MatrixLayout("P")
layout.map("P", spec)
layout.display()


## 8. Layout Multi-Gráfico - Dashboard Completo


In [ ]:
# Crear dashboard con múltiples gráficos
layout = MatrixLayout("""
SH
BL
""", figsize=(800, 600))

# Scatter plot (S)
scatter_spec = ChartRegistry.get('scatter').get_spec(
    data=df,
    x_col='edad',
    y_col='salario',
    category_col='departamento',
    interactive=True
)
layout.map("S", scatter_spec)

# Histograma (H)
hist_spec = ChartRegistry.get('histogram').get_spec(
    data=df,
    value_col='salario',
    bins=15,
    interactive=True
)
layout.map("H", hist_spec)

# Bar chart (B)
bar_spec = ChartRegistry.get('bar').get_spec(
    data=df,
    category_col='departamento',
    value_col='ventas',
    interactive=True
)
layout.map("B", bar_spec)

# Line chart (L)
df_sorted = df.sort_values('experiencia')
line_spec = ChartRegistry.get('line').get_spec(
    data=df_sorted,
    x_col='experiencia',
    y_col='productividad',
    interactive=True
)
layout.map("L", line_spec)

layout.display()


## 9. Usando Preparadores de Datos Directamente


In [ ]:
# Validar datos antes de preparar
from BESTLIB.data import validate_scatter_data, prepare_scatter_data

# Validar
validate_scatter_data(df, x_col='edad', y_col='salario')
print("✅ Datos válidos")

# Preparar datos
processed, original = prepare_scatter_data(
    df,
    x_col='edad',
    y_col='salario',
    category_col='departamento'
)

print(f"\nDatos procesados: {len(processed)} puntos")
print(f"Primer punto: {processed[0]}")


## 10. Listar Gráficos Disponibles


In [ ]:
# Listar todos los gráficos registrados
chart_types = ChartRegistry.list_types()
print("Gráficos disponibles:")
for chart_type in chart_types:
    print(f"  - {chart_type}")


## Resumen

Este notebook demuestra:

1. ✅ Uso de ChartRegistry para obtener gráficos
2. ✅ Generación de specs usando la nueva API modular
3. ✅ Diferentes tipos de gráficos (scatter, bar, histogram, boxplot, pie, line, heatmap)
4. ✅ Layouts multi-gráfico
5. ✅ Uso de preparadores de datos directamente
6. ✅ Listado de gráficos disponibles

La nueva estructura modular permite:
- **Extensibilidad**: Agregar nuevos gráficos sin modificar código existente
- **Modularidad**: Usar componentes individuales (preparadores, validadores)
- **Mantenibilidad**: Código organizado por responsabilidades
- **Compatibilidad**: API antigua sigue funcionando
